In [1]:
'''
from math import log
nBoost = 10
learners = [None] * nBoost # storage for ensemble of models
errors = [0.0] * nBoost
weights = [1.0] * 10000 # and weights of each learner
alpha = [0.0] * nBoost #idk what alpha is used for...

mu = np.mean(Ytr)
dY = Ytr - mu

for i in range(nBoost):
    learners[i] = ml.bayes.gaussClassify()
    learners[i].train(Xtr, Ytr, wts=weights)
    Yhat = learners[i].predict(Xtr)
    print learners[i].err(Xtr, Ytr)
    e = np.dot(weights, (Ytr!=Yhat))/10000
    alpha[i] = 0.5 * log((1-e)/e)
    weights *= np.exp(-alpha[i] * Ytr * Yhat)
    weights /= weights.sum()
'''

'\nfrom math import log\nnBoost = 10\nlearners = [None] * nBoost # storage for ensemble of models\nerrors = [0.0] * nBoost\nweights = [1.0] * 10000 # and weights of each learner\nalpha = [0.0] * nBoost #idk what alpha is used for...\n\nmu = np.mean(Ytr)\ndY = Ytr - mu\n\nfor i in range(nBoost):\n    learners[i] = ml.bayes.gaussClassify()\n    learners[i].train(Xtr, Ytr, wts=weights)\n    Yhat = learners[i].predict(Xtr)\n    print learners[i].err(Xtr, Ytr)\n    e = np.dot(weights, (Ytr!=Yhat))/10000\n    alpha[i] = 0.5 * log((1-e)/e)\n    weights *= np.exp(-alpha[i] * Ytr * Yhat)\n    weights /= weights.sum()\n'

In [2]:
'''
# Final classifier
predict = np.zeros((10000,))
for i in range(nBoost):
    predict += alpha[i] * learners[i].predict(Xva)
    
for i in range(len(predict)):
    if i == 0:
        predict[i] = -1
predict = np.sign(predict)
for i in range(len(predict)):
    if i == -1:
        predict[i] = 0
        
print 'error is', 1.0*sum(predict != Yva)/10000
'''

"\n# Final classifier\npredict = np.zeros((10000,))\nfor i in range(nBoost):\n    predict += alpha[i] * learners[i].predict(Xva)\n    \nfor i in range(len(predict)):\n    if i == 0:\n        predict[i] = -1\npredict = np.sign(predict)\nfor i in range(len(predict)):\n    if i == -1:\n        predict[i] = 0\n        \nprint 'error is', 1.0*sum(predict != Yva)/10000\n"

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml


X = np.genfromtxt("data/X_train.txt")
Y = np.genfromtxt("data/Y_train.txt")
Xte = np.genfromtxt('data/X_test.txt')

'''
Xtr = X[10000:20000]
Xva = X[20000:30000]
Ytr = Y[10000:20000]
Yva = Y[20000:30000]
'''

Xtr, Xva, Ytr, Yva = ml.splitData(X, Y, 0.80)

In [2]:
def get_err_rate(yval, ypred):
    err_count = 0
    for i in range(len(yval)):
        if not(yval[i] == ypred[i]):
            err_count+=1
    return 1.0*err_count/len(yval)

In [3]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn import gaussian_process
from sklearn import linear_model

In [10]:
dt = tree.DecisionTreeClassifier(max_depth=1)
ada_clf = AdaBoostClassifier(base_estimator=dt, n_estimators=50)
ada_clf.fit(Xtr, Ytr)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=None)

In [11]:
ada_yhat = ada_clf.predict_proba(Xtr)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, ada_yhat[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print roc_auc

0.678114198594


In [12]:
ada_yhat = ada_clf.predict_proba(Xva)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, ada_yhat[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print roc_auc

0.672908172804


In [13]:
# Now output a file with two columns, a row ID and a confidence in class 1:
ada_yhat = ada_clf.predict_proba(Xte)
np.savetxt('Yhat_adaboost_decisiontree.txt',
np.vstack( (np.arange(len(ada_yhat)) , ada_yhat[:,1]) ).T,
'%d, %.2f',header='ID,Prob1',comments='',delimiter=',');